This notebook is used to 
- compute the SHAP value change for DT
- plot the SHAP value maps
- notice that the prob in the title is that f(x) becomes committor.

In [ ]:
import xarray as xr
import numpy as np
import cartopy
from cartopy import crs as ccrs
import matplotlib 
matplotlib.rcParams["font.size"] = 12
from matplotlib import pyplot as plt
from os.path import join, exists
from os import mkdir
import scipy
import netCDF4
import sklearn
import sys
import matplotlib.ticker as mticker
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter,
                                LatitudeLocator, LongitudeLocator)
%matplotlib inline
import matplotlib.path as mpath
import importlib.util
import MM_util_AI
import MM_utilplot
import warnings
import pickle

warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*")
import shap
import tensorflow as tf
from sklearn.metrics import confusion_matrix,recall_score,precision_score
import matplotlib.colors as colors
import os

spec = importlib.util.spec_from_file_location("MM_dataprepare", \
                        "/scratch/hz1994/blocking/MMmodel/MMmodel/notebooks/MM_dataprepare.py")
MM_dataprepare = importlib.util.module_from_spec(spec)
sys.modules["module.name"] = MM_dataprepare
spec.loader.exec_module(MM_dataprepare)

spec = importlib.util.spec_from_file_location("MM_utilblocking", \
                        "/scratch/hz1994/blocking/MMmodel/MMmodel/notebooks/MM_utilblocking.py")
MM_utilblocking = importlib.util.module_from_spec(spec)
sys.modules["module.name"] = MM_utilblocking
spec.loader.exec_module(MM_utilblocking)

 
            
with open("/scratch/hz1994/blocking/data_MMmodel/filepath.txt","r") as fi:
    for ln in fi:
        if ln.startswith("dimensionalized_filepath"):
            dim_path=ln.strip().split('\t')[1]
        if ln.startswith("nondimensionalized_filepath"):
            nondim_path=ln.strip().split('\t')[1]
        if ln.startswith("code_filepath"):
            code_path=ln.strip().split('\t')[1]            
        if ln.startswith("DGindex_filepath"):
            DGindex_path=ln.strip().split('\t')[1]  
        if ln.startswith("conditionedT_filepath" ):
            train_path=ln.strip().split('\t')[1]
        if ln.startswith("model_filepath" ):
            models_path=ln.strip().split('\t')[1]
        if ln.startswith("fig_filepath" ):
            fig_path=ln.strip().split('\t')[1] 
print(dim_path)
print(nondim_path)
print(code_path)
print(DGindex_path)
print(train_path)
print(models_path)
print(fig_path)
train_path_setA=train_path+'T/'
models_path_setA=models_path+'T/'
import tensorflow as tf


# Prepare data

In [ ]:
Duration=5
cnnsize="normal"
data_amount="1000.0k"
random_seed=30
epsilon=0
trainable_layer_number=7  
epoch_DT=3  
class_imb_Method=""

X=np.load("/scratch/hz1994/blocking/data_era5/"+"test_data_1940-2022.npy")  
Ysparse=np.load("/scratch/hz1994/blocking/data_era5/"+"test_labels_1940-2022_T%d.npy"%Duration)
Y=np.zeros((Ysparse.size,2)).astype(bool)
Y[:,1][Ysparse==1]=True  #blocking
Y[:,0][Ysparse==0]=True
latitudes = np.load(dim_path+'dataX_lat.npy')
longitudes = np.load(dim_path+'dataX_lon.npy')
X_dim=X[0].size

# Load model before transfer learning

In [ ]:
tf.config.experimental_run_functions_eagerly(False)
if cnnsize=="smaller":
    DT_model = MM_util_AI.make_s_model((18,90,3))
elif cnnsize=="smaller_smaller":
    DT_model = MM_util_AI.make_ss_model((18,90,3))
elif cnnsize=="normal":
    DT_model = MM_util_AI.make_model((18,90,3))


# Pick the data we want to make the plot: TP  

In [ ]:
TP_consistent=np.ones(Y.shape[0]).astype(bool)
for random_seed in range(10): 
    path="/scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN"+class_imb_Method+"/extreme_%ddaysblocking/%s/"%(Duration,cnnsize)\
                +"random_initCNN_%d/"%(random_seed)
    for num in range(10): 
        weightpath=path+"%d/"%num +"cp-%04d.ckpt"%epoch_DT 
        DT_model.load_weights(weightpath)
        predictions = DT_model.predict(X,verbose=None)
        pred_DT=(predictions[:,0]<predictions[:,1])
        TP_consistent=np.logical_and(TP_consistent,np.logical_and(pred_DT, Ysparse)) 

In [ ]:
plot1=[]
for random_seed in range(10):
    path="/scratch/hz1994/blocking/data_MMmodel/CNNmodels/T/era5_trainCNN"+class_imb_Method+"/extreme_%ddaysblocking/%s/"%(Duration,cnnsize)\
                +"random_initCNN_%d/"%(random_seed)
    for num in range(10): 
        DT_shap=np.load(path+"%d/"%num+"shap_values_era5_epoch%d.npy"%epoch_DT)[TP_consistent,1,0]
        plot1.append(DT_shap)  
plot1=np.array(plot1)
plot1_mean=plot1.mean(axis=(0,1))
save_path= "/scratch/hz1994/blocking/MMmodel/MMmodel/notebooks/All_paper_Jupyter_script/"
np.save(save_path+"shap_DT",plot1_mean)